In [1]:
pip install efficientnet -q

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import efficientnet.tfkeras as efn
from kaggle_datasets import KaggleDatasets

import tensorflow as tf
from tensorflow.keras.applications import Xception
from keras.layers import Dense, Flatten, Activation, Conv2D, MaxPooling2D, Dropout, Conv2D,MaxPooling2D,GlobalAveragePooling2D
from keras.optimizers import Adam
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import optimizers

from sklearn.metrics import roc_auc_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('gs://'):
    print(os.path.join(dirname))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
from tensorflow.python.client import device_lib

print(tf.test.gpu_device_name())
print(device_lib.list_local_devices())

print("GPU Available: ", tf.test.is_gpu_available())

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

/device:GPU:0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17187802288787944535
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15479566528
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11928919760620582120
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:00:04.0, compute capability: 7.0"
]
GPU Available:  True
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [4]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(256, 256), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.2)
        img = tf.image.random_contrast(img, 0.8, 1.2)
        img = tf.image.random_hue(img, 0.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

Variables and configurations
---

In [5]:
# COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16
# GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)
print(BATCH_SIZE)

Running on 1 replicas
16


In [6]:
XCEPTION = False
EFFICIENT = True

if XCEPTION:
    IMAGE_SIZE = 256#800
    model_name = "xception"
if EFFICIENT:
    IMAGE_SIZE = 640
    model_name = "efficient"
    
USE_TEST_FOLD = True

Preparing dataset
---

Loading and preprocess CSVs

In [7]:
# load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
extras_PATH = 'gs://extras-entrenamiento/'
datos_PATH = os.getcwd() + '/train/'#'gs://ranzcr-data/'
df = pd.read_csv(extras_PATH + 'train_folds.csv')

# paths = load_dir + "train/" + df['StudyInstanceUID'] + '.jpg'
paths = datos_PATH + df['StudyInstanceUID'] + '.jpg'

# sub_df = pd.read_csv(load_dir + 'sample_submission.csv')

# test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'
# test_paths = GCS_DS_PATH + "/test/" + sub_df['StudyInstanceUID'] + '.jpg'

# Get the multi-labels
label_cols = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal', 
                 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                 'Swan Ganz Catheter Present']
labels = df[label_cols].values

In [8]:
# select folds here
FOLDS_PATH = "../input/split-k-folds/"

CURRENT_FOLD = 0# sus datos se usan para validar, el resto para entrenar

dfx = pd.read_csv(os.path.join(extras_PATH, "train_folds.csv"))
print("Folds are: ", sorted(dfx.fold.unique()))

targets = [
    "ETT - Abnormal",
    "ETT - Borderline",
    "ETT - Normal",
    "NGT - Abnormal",
    "NGT - Borderline",
    "NGT - Incompletely Imaged",
    "NGT - Normal",
    "CVC - Abnormal",
    "CVC - Borderline",
    "CVC - Normal",
    "Swan Ganz Catheter Present",
]
df_train = dfx[dfx.fold != CURRENT_FOLD].reset_index(drop=True)
df_valid = dfx[dfx.fold == CURRENT_FOLD].reset_index(drop=True)

print(df_train.shape)
print(df_valid.shape)

# Para validar y asignar pesos de los modelos
if USE_TEST_FOLD:
    test_fold = 0
    df_train = df_train[df_train.fold_test != test_fold].reset_index(drop=True)
    df_valid = df_valid[df_valid.fold_test != test_fold].reset_index(drop=True)
    print(df_train.shape)
    print(df_valid.shape)

df_valid.head()

Folds are:  [0, 1, 2, 3, 4]
(24080, 15)
(6003, 15)
(23091, 15)
(5747, 15)


,StudyInstanceUID,ETT - Abnormal,ETT - Borderline,ETT - Normal,NGT - Abnormal,NGT - Borderline,NGT - Incompletely Imaged,NGT - Normal,CVC - Abnormal,CVC - Borderline,CVC - Normal,Swan Ganz Catheter Present,PatientID,fold,fold_test
0,1.2.826.0.1.3680043.8.498.71008408169501434503...,0,0,0,0,0,0,0,0,0,1,0,7d2c4c9e3,0,15
1,1.2.826.0.1.3680043.8.498.12874505746046378224...,0,0,0,0,0,0,0,0,0,1,0,d46ba6d06,0,11
2,1.2.826.0.1.3680043.8.498.36104631337783803705...,0,0,0,0,0,0,0,0,0,1,0,0e93aeeb4,0,14
3,1.2.826.0.1.3680043.8.498.44301685598654680107...,0,0,1,0,0,0,1,0,0,1,0,4099d5872,0,20
4,1.2.826.0.1.3680043.8.498.37913195819499413602...,0,0,1,0,0,1,0,0,0,1,0,a01d16803,0,14


In [9]:
# IMAGE_PATH = GCS_DS_PATH + "/train/"

train_image_paths = [
    os.path.join(datos_PATH, x + ".jpg") for x in df_train.StudyInstanceUID.values
]
valid_image_paths = [
    os.path.join(datos_PATH, x + ".jpg") for x in df_valid.StudyInstanceUID.values
]

train_targets = df_train[targets].values
valid_targets = df_valid[targets].values

In [10]:
#B7 = 600

decoder = build_decoder(with_labels=True, target_size=(IMAGE_SIZE, IMAGE_SIZE))
test_decoder = build_decoder(with_labels=False, target_size=(IMAGE_SIZE, IMAGE_SIZE))

train_dataset = build_dataset(
    train_image_paths, train_targets, bsize=BATCH_SIZE, decode_fn=decoder
)

valid_dataset = build_dataset(
    valid_image_paths, valid_targets, bsize=BATCH_SIZE, decode_fn=decoder,
    repeat=False, shuffle=False, augment=False
)

# test_dataset = build_dataset(
#     test_paths, cache=False, bsize=BATCH_SIZE, decode_fn=test_decoder,
#     repeat=False, shuffle=False, augment=False
# )

Modeling
----

Binary cross-entropy is for multi-label classifications, whereas categorical cross entropy is for multi-class classification where each example belongs to a single class.

In [11]:
LR = 3e-4

Efficient model B7
---

In [12]:
if EFFICIENT:
    n_labels = labels.shape[1]

    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(
                input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                weights='imagenet',
                include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            #tf.keras.layers.Dense(512, activation='sigmoid'),
            #tf.keras.layers.PReLU(),
            #tf.keras.layers.Dropout(rate=0.5),
            tf.keras.layers.Dense(n_labels, activation='sigmoid')
        ])
        model.compile(
            optimizer=optimizers.Adam(lr=LR),
            loss='binary_crossentropy',
            metrics=[tf.keras.metrics.AUC(multi_label=True)])
        model.summary()

258441216/258434480 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Functional) (None, 20, 20, 2560)      64097680  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2560)              0         
_________________________________________________________________
dense (Dense)                (None, 11)                28171     
Total params: 64,125,851
Trainable params: 63,815,131
Non-trainable params: 310,720
_________________________________________________________________


XCEPTION Model
---

In [13]:
if XCEPTION:
    
    n_labels = labels.shape[1]
    
    with strategy.scope():
        net = Xception(include_top=False,
                       input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                       weights='imagenet')
        x = net.output
        x = GlobalAveragePooling2D()(x)
        #x = Dropout(0.4)(x)
        output = Dense(n_labels, activation='sigmoid')(x)
        model = Model(inputs=net.input, outputs=output)
        model.compile(optimizers.Adam(lr=LR),
                      loss='binary_crossentropy',
                      metrics=[tf.keras.metrics.AUC(multi_label=True)])

In [14]:
steps_per_epoch = df_train.shape[0] // BATCH_SIZE

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model_fold' + str(CURRENT_FOLD) + model_name + '.h5', save_best_only=True, monitor='val_loss', mode='min')

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", patience=0, min_lr=1e-6, mode='min')

early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 1e-4, patience = 5, mode = 'min', 
                    restore_best_weights = True, verbose = 0)

In [ ]:
history = model.fit(
    train_dataset, 
    epochs=10,#25
    verbose=2,
    callbacks=[checkpoint, lr_reducer],
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_dataset)

Epoch 1/10


In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('history_fold'+ str(CURRENT_FOLD) + '.csv')

Validation with test data
---

In [ ]:
if USE_TEST_FOLD:
    # filtramos
    test_fold = 0
    df_test_fold = dfx[dfx.fold_test == test_fold].reset_index(drop=True)
    print(df_test_fold.shape)
    # creamos el dataset
    test_image_paths = [os.path.join(IMAGE_PATH, x + ".jpg") for x in df_test_fold.StudyInstanceUID.values]
    test_targets = df_test_fold[targets].values
    test_dataset = build_dataset(test_image_paths,
                                 test_targets,
                                 bsize=BATCH_SIZE,
                                 decode_fn=decoder,
                                 repeat=False, shuffle=False, augment=False)
    # predecimos ;)
    y_pred = model.predict(test_dataset, verbose=1)
    y_real = df_test_fold.iloc[:, 1:13]
    acc_fold = np.mean([roc_auc_score(y_real.iloc[:, i], y_pred[:, i]) for i in range(11)])
    
    print("Final ACC for this fold:", acc_fold)
    
    bce = tf.keras.losses.BinaryCrossentropy(from_logits = True)
    loss = np.mean([bce(y_real.iloc[:, i], y_pred[:, i]) for i in range(11)])
    print('Loss: ', loss.numpy()) 